## Импорты

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
from tqdm import tqdm_notebook, trange, tqdm
import pickle
import numpy as np
from IPython.display import clear_output

from collections import defaultdict

In [5]:
base_path = '/Users/dentarasov/Yandex.Disk.localized/current/vkr'
data_path = base_path + '/after_preprocess/all_inputs'

with open(data_path, 'rb') as f:
    data = pickle.load(f)


In [6]:
data.shape

(695687, 100)

In [7]:
split_num = 600000
train, test = data[:split_num], data[split_num:]

In [8]:
N = 3

In [9]:
ngrams_dict = defaultdict(int)

In [31]:
# for seq in tqdm_notebook(train):
#     for i in range(len(seq) - N):
#         ngrams_dict[tuple(seq[i:i+N])] += 1

## Что стоит сделать
* Убрать `<PAD>`

In [ ]:
def filter_most_popular_ngrams(ngrams_dict):
    # ngram_list: [[(1, 231, 12), 5], ...]
    ngrams_list = list(ngrams_dict.items())
    def sort_fun(ngram):
        return ngram[0][:2], ngram[1]
    ngrams_list = sorted(ngrams_list, key=sort_fun, reverse=True)
    prev = None
    new_ngrams_dict = {}
    for ngram in tqdm(ngrams_list):
        cur = ngram[0][:2]
        if prev != cur:
            new_ngrams_dict[tuple(cur)] = ngram[0][2]
        prev = cur
    return new_ngrams_dict

In [ ]:
# new_ngrams_dict = filter_most_popular_ngrams(ngrams_dict)

In [11]:
# with open('./ngrams_dict.pickle', 'wb') as f:
#     pickle.dump(new_ngrams_dict, f)

with open('./ngrams_dict.pickle', 'rb') as f:
    new_ngrams_dict = pickle.load(f)

In [12]:
list(new_ngrams_dict.items())[:10]

[((79583, 14), 3),
 ((79583, 9), 2),
 ((79583, 5), 14),
 ((79583, 3), 2),
 ((79583, 2), 2),
 ((79581, 4), 1),
 ((79581, 3), 42671),
 ((79581, 2), 15),
 ((79580, 14), 7),
 ((79580, 9), 13)]

In [13]:
def build_vocab():
    # load pre-computed vocab
    with open(base_path + '/mapping.map', 'rb') as f:
        word_to_id = pickle.load(f)
    id_to_word = dict([(v, k) for (k, v) in word_to_id.items()])
    return word_to_id, id_to_word

word_to_id, id_to_word = build_vocab()


In [34]:
def calc_quality(ngrams_dict):
    tp = 0
    count = 0
    for seq in tqdm(test):
        for i in range(len(seq)-N):
            # to avoid considering empty tokens
            # and non-identifiers
            if seq[i+1] != 0 and seq[i+2] != 0 and id_to_word[seq[i+2]].isidentifier():
                if (seq[i], seq[i+1]) in ngrams_dict \
                        and seq[i+2] == ngrams_dict[(seq[i], seq[i+1])]:
                    tp += 1
                count += 1
    return tp / count
        

In [42]:
accuracy = calc_quality(new_ngrams_dict)
print('''Accuracy: {:.4f}
Time needed for one prediction is just accessing the dict element, so it\'s around ~12 sec / ~96000 = {:.7f} (info from tqdm counter)'''\
.format(accuracy, 12. / 96000))


100%|██████████| 95687/95687 [00:12<00:00, 7505.28it/s]

Accuracy: 0.1552
Time needed for one prediction is just accessing the dict element, so it's around ~12 sec / ~96000 = 0.0001250 (info from tqdm counter)


In [43]:
test.shape[0] * (test.shape[1] - N)

9281639